# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
quantify=data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})

In [5]:
quantify.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
pivot = pd.pivot_table(data, values='Quantity', index='ProductName',
                    columns='CustomerID', aggfunc=np.sum, fill_value=0)

In [7]:
pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
distances=squareform(pdist(pivot.T, 'euclidean'))

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [9]:
customers = pd.DataFrame(1/(1 + distances), 
                         index=pivot.columns, columns=pivot.columns)
customers.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [10]:
similar_customers=list(customers[33].sort_values(ascending=False)[1:6].index)

In [11]:
similar_customers

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [12]:
sim_customers_records=quantify.loc[similar_customers]

In [13]:
sim_customers_records

Quantity
CustomerID ProductName                            
264        Apricots - Halves                     1
           Apricots Fresh                        1
           Bacardi Breezer - Tropical            1
           Bagel - Plain                         1
           Banana - Leaves                       1
...                                            ...
3305       Wine - Ej Gallo Sierra Valley         2
           Wine - Pinot Noir Latour              1
           Wine - Redchard Merritt               1
           Wine - White Cab Sauv.on              1
           Yogurt - French Vanilla               1

[270 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [14]:
products=sim_customers_records.groupby(['ProductName']).agg({'Quantity':sum}).sort_values(by='Quantity',ascending=False)

In [15]:
products

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [16]:
merged=products.merge(right=pivot,how='left',on='ProductName')

In [17]:
merged_33=merged[['Quantity',33]].head(10)

In [18]:
recomm_products=list(merged_33[merged_33[33]==0].head(5).index)

In [19]:
recomm_products

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [20]:
recommendations={}
customersID=sorted(list(data['CustomerID'].unique()))

In [21]:
for customer in customersID:
    similar_customers=list(customers[customer].sort_values(ascending=False)[1:6].index)
    sim_customers_records=quantify.loc[similar_customers]
    products=sim_customers_records.groupby(['ProductName']).agg({'Quantity':sum}).sort_values(by='Quantity',ascending=False)
    merged=products.merge(right=pivot,how='left',on='ProductName')
    merged_customerID=merged[['Quantity',customer]].head(10)
    customer_reco={customer:list(merged_customerID[merged_customerID[customer]==0].head(5).index)}
    recommendations.update(customer_reco)


In [22]:
recommendations[33]

['Butter - Unsalted',
 'Wine - Ej Gallo Sierra Valley',
 'Soup - Campbells Bean Medley',
 'Wine - Blue Nun Qualitatswein',
 'Chicken - Soup Base']

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [23]:
customers_recos=pd.DataFrame.from_dict(recommendations,orient='index',columns=['Product1','Product2','Product3','Product4','Product5'])

In [24]:
customers_recos.head(5)

,Product1,Product2,Product3,Product4,Product5
33,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Soup - Campbells Bean Medley,Wine - Blue Nun Qualitatswein,Chicken - Soup Base
200,Soup - Campbells Bean Medley,Muffin - Carrot Individual Wrap,Bay Leaf,Pork - Kidney,Wanton Wrap
264,Soupfoamcont12oz 112con,Wine - Two Oceans Cabernet,Bread - Italian Roll With Herbs,"Veal - Inside, Choice","Fish - Scallops, Cold Smoked"
356,"Veal - Inside, Choice",Lamb - Ground,Pomello,Butter - Unsalted,"Tart Shells - Sweet, 4"
412,Olive - Spread Tapenade,Sprouts - Baby Pea Tendrils,Wine - Blue Nun Qualitatswein,"Pepper - Black, Whole",Soup - Campbells Bean Medley


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [26]:
distances_cos=squareform(pdist(pivot.T, 'cosine'))
customers_cos = pd.DataFrame(1/(1 + distances_cos), 
                         index=pivot.columns, columns=pivot.columns)
customers_cos.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.530003,0.563492,0.543546,0.542393,0.541054,0.522016,0.530783,0.533096,0.535293,...,0.530306,0.517987,0.527662,0.511471,0.534181,0.513809,0.528795,0.520208,0.529481,0.527142
200,0.530003,1.000000,0.537166,0.535098,0.521226,0.519193,0.556945,0.516729,0.536350,0.541451,...,0.545224,0.533067,0.555246,0.543554,0.530308,0.521925,0.573177,0.537284,0.526167,0.527280
264,0.563492,0.537166,1.000000,0.531963,0.550168,0.525368,0.538967,0.514932,0.537046,0.524193,...,0.546880,0.525550,0.535062,0.531806,0.553085,0.560225,0.548538,0.538340,0.554929,0.527142
356,0.543546,0.535098,0.531963,1.000000,0.525982,0.523986,0.536792,0.540890,0.516901,0.519482,...,0.536321,0.520565,0.533119,0.534066,0.528874,0.530406,0.520138,0.536202,0.544727,0.535974
412,0.542393,0.521226,0.550168,0.525982,1.000000,0.534073,0.542308,0.547743,0.530459,0.543050,...,0.527903,0.565369,0.528847,0.529278,0.541674,0.536686,0.519628,0.538710,0.546943,0.552636


In [55]:
recommendations_cos={}

In [27]:
for customer in customersID:
    similar_customers=list(customers_cos[customer].sort_values(ascending=False)[1:6].index)
    sim_customers_records=quantify.loc[similar_customers]
    products=sim_customers_records.groupby(['ProductName']).agg({'Quantity':sum}).sort_values(by='Quantity',ascending=False)
    merged=products.merge(right=pivot,how='left',on='ProductName')
    merged_customerID=merged[['Quantity',customer]].head(10)
    customer_reco={customer:list(merged_customerID[merged_customerID[customer]==0].head(5).index)}
    recommendations.update(customer_reco)

In [28]:
customers_recos_cos=pd.DataFrame.from_dict(recommendations,orient='index',columns=['Product1','Product2','Product3','Product4','Product5'])

In [29]:
customers_recos_cos.head(5)

,Product1,Product2,Product3,Product4,Product5
33,Knife Plastic - White,"Soup - Campbells, Beef Barley",Onions - Cippolini,Tea - Herbal Sweet Dreams,None
200,Longos - Grilled Salmon With Bbq,Snapple Lemon Tea,General Purpose Trigger,None,None
264,Pickerel - Fillets,"Water - Mineral, Natural",Snapple - Iced Tea Peach,Wine - Ej Gallo Sierra Valley,French Pastry - Mini Chocolate
356,Bread - English Muffin,Olive - Spread Tapenade,Bagel - Plain,Pork - Hock And Feet Attached,None
412,"Salsify, Organic",Durian Fruit,Wine - Hardys Bankside Shiraz,Bread - Raisin Walnut Oval,None
